In [ ]:
# Import

from selenium import webdriver
from selenium.webdriver.common.by import By
from dataclasses import dataclass
import time

In [94]:
# Faça scraping para obter os 250 séries com as maiores avaliações do IMDB. Devem ser
# obtidos: Título, Ano de estreia, Número Total de Episódios, Nota do IMDB e o link para
# página da série.

driver = webdriver.Chrome()
driver.get("https://www.imdb.com/")
driver.maximize_window()

In [ ]:
menu = driver.find_element(By.XPATH, r"/html/body/div[2]/nav/div[2]/div[1]/label")
time.sleep(2)
menu.click()
time.sleep(2)
link_250series = driver.find_element(By.LINK_TEXT, r'Top 250 TV Shows')
link_250series.click()

tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
lista_filmes = tag_ul.find_elements(By.TAG_NAME, "li")

# for filme in lista_filmes:
#     print(filme.find_element(By.CLASS_NAME, "ipc-title__text").text)

In [13]:
@dataclass
class Filme:
    titulo: str
    ano: str
    n_episodios: str
    classificacao: float
    pagina: str

# lista_filmes[0]
# pagina = lista_filmes[0].find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
# pagina

def cria_filme(imdb_li_tag):
    titulo = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-title__text").text.split(' ', 1)[1]
    ano = imdb_li_tag.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[1]').text
    n_episodios = imdb_li_tag.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[2]').text
    texto_classificacao = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text
    pagina = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
        
    return Filme(titulo, ano, n_episodios, texto_classificacao, pagina)

lista_de_filmes = []
for filme_tag in lista_filmes:
    filme = cria_filme(filme_tag)
    lista_de_filmes.append(filme)

lista_de_filmes

[Filme(titulo='Breaking Bad', ano='2008–2013', n_episodios='62 eps', classificacao='9.5', pagina='https://www.imdb.com/title/tt0903747/?ref_=chttvtp_t_1'),
 Filme(titulo='Planet Earth II', ano='2016', n_episodios='6 eps', classificacao='9.4', pagina='https://www.imdb.com/title/tt5491994/?ref_=chttvtp_t_2'),
 Filme(titulo='Planet Earth', ano='2006', n_episodios='11 eps', classificacao='9.4', pagina='https://www.imdb.com/title/tt0795176/?ref_=chttvtp_t_3'),
 Filme(titulo='Band of Brothers', ano='2001', n_episodios='10 eps', classificacao='9.4', pagina='https://www.imdb.com/title/tt0185906/?ref_=chttvtp_t_4'),
 Filme(titulo='Chernobyl', ano='2019', n_episodios='5 eps', classificacao='9.3', pagina='https://www.imdb.com/title/tt7366338/?ref_=chttvtp_t_5'),
 Filme(titulo='The Wire', ano='2002–2008', n_episodios='60 eps', classificacao='9.3', pagina='https://www.imdb.com/title/tt0306414/?ref_=chttvtp_t_6'),
 Filme(titulo='Avatar: The Last Airbender', ano='2005–2008', n_episodios='62 eps', cla

In [ ]:
# Faça scraping das páginas específicas dos 250 filmes obtidos no item anterior. Obte-
# nha dessa página a popularidade e a listagem do elenco principal (incluindo nome do
# ator/atriz e da personagem).

@dataclass
class Ator:
    serie: str
    nome: str
    personagem: str

def coletar_ator(elenco, driver):
    atores = []
    nome_serie = driver.find_element(By.CLASS_NAME, "hero__primary-text").text
    personagens = elenco.find_elements(By.XPATH, './div')
    
    for personagem in personagens:
        div_info = personagem.find_element(By.XPATH, './div[2]')
        nome_ator = div_info.find_element(By.XPATH, './a').text
        nome_personagem = div_info.find_element(By.XPATH, './div/ul/li/a/span').text
        
        ator = Ator(nome_serie, nome_ator, nome_personagem)
        atores.append(ator)
        print(ator)
    
    return atores

In [ ]:
lista_de_atores = []

for filme in lista_de_filmes:
    driver.get(filme.pagina)
    time.sleep(3)
    
    elenco = driver.find_element(By.XPATH, "/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]")
    atores = coletar_ator(elenco, driver)
    lista_de_atores.extend(atores)

In [95]:
@dataclass
class Ator:
    serie: str
    nome: str
    personagem: str

def coletar_ator(elenco, driver):
    atores = []
    nome_serie = driver.find_element(By.CLASS_NAME, "hero__primary-text").text
    
    personagens = elenco.find_elements(By.XPATH, './div')
    
    for personagem in personagens:
        try:
            div_info = personagem.find_element(By.XPATH, './div[2]')
            nome_ator = div_info.find_element(By.XPATH, './a').text
            nome_personagem = div_info.find_element(By.XPATH, './div/ul/li/a/span').text
            
            ator_obj = Ator(nome_serie, nome_ator, nome_personagem)
            atores.append(ator_obj)
            print(ator_obj)
        
        except Exception as e:
            print(f"[{nome_serie}] Erro ao processar personagem: {e}")
            continue
    
    return atores

lista_de_atores = []

for filme in lista_de_filmes:
    driver.get(filme.pagina)
    time.sleep(3)
    
    try:
        elenco = driver.find_element(By.XPATH, "/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]")
        atores = coletar_ator(elenco, driver)
        lista_de_atores.extend(atores)
    
    except Exception as e:
        print(f"Erro ao acessar elenco do filme {filme}: {e}")

Ator(serie='Breaking Bad', nome='Bryan Cranston', personagem='Walter White')
Ator(serie='Breaking Bad', nome='Aaron Paul', personagem='Jesse Pinkman')
Ator(serie='Breaking Bad', nome='Anna Gunn', personagem='Skyler White')
Ator(serie='Breaking Bad', nome='Betsy Brandt', personagem='Marie Schrader')
Ator(serie='Breaking Bad', nome='RJ Mitte', personagem='Walter White, Jr.')
Ator(serie='Breaking Bad', nome='Dean Norris', personagem='Hank Schrader')
Ator(serie='Breaking Bad', nome='Bob Odenkirk', personagem='Saul Goodman')
Ator(serie='Breaking Bad', nome='Steven Michael Quezada', personagem='Steven Gomez')
Ator(serie='Breaking Bad', nome='Jonathan Banks', personagem='Mike Ehrmantraut')
Ator(serie='Breaking Bad', nome='Giancarlo Esposito', personagem='Gus Fring')
Ator(serie='Breaking Bad', nome='Charles Baker', personagem='Skinny Pete')
Ator(serie='Breaking Bad', nome='Jesse Plemons', personagem='Todd')
Ator(serie='Breaking Bad', nome='Christopher Cousins', personagem='Ted Beneke')
Ator(se

In [96]:
series_distintas = set(ator.serie for ator in lista_de_atores)
print(f"Número de séries distintas: {len(series_distintas)}")

Número de séries distintas: 244
